<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240201-%E5%BE%97%E5%88%B02470%E6%A0%B7%E6%9C%AC%E7%9A%842022NDRE%E6%97%B6%E9%97%B4%E5%BA%8F%E5%88%97(3.21-8.23).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [2]:
import os
import pandas as pd
import math

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
point = ee.FeatureCollection("projects/ee-grn/assets/randomsample_2022_1000")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


In [5]:
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')


# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [6]:
# 5天序列NDRE
interval = 5
increment = 'day'
start = '2022-03-21'
startDate = ee.Date(start)
secondDate = startDate.advance(interval, increment).millis()
increase = secondDate.subtract(startDate.millis())
list5d = ee.List.sequence(startDate.millis(), ee.Date('2022-08-23').millis(), increase)

def add_time(date):
  return ee.Image(0).mask(0).rename('x').set('system:time_start',ee.Date(date).millis())

dailyMean =  ee.ImageCollection.fromImages(list5d.map(add_time))

def add_allbands(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  timeRadians1 = ee.Image(years).multiply(2 * math.pi)
  return image.addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1)).addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

s2makeup = dailyMean.map(add_allbands)

def add_fittedNDRE(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('NDRE'))

fitteds2 = s2makeup.map(add_fittedNDRE)

HANTS = fitteds2.select('NDRE')

In [7]:
# NDRE时间序列合成
image_agrgt = HANTS.toBands()
print(image_agrgt.bandNames().getInfo())

['0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE']


In [8]:
image_agrgt_clip = image_agrgt.clip(roi)

In [9]:
samples_datesets=image_agrgt_clip.sampleRegions(collection=point,properties=['id'],scale=10,tileScale=8)

In [10]:
cols = ['id', '0_NDRE', '1_NDRE', '2_NDRE', '3_NDRE', '4_NDRE', '5_NDRE', '6_NDRE', '7_NDRE', '8_NDRE', '9_NDRE', '10_NDRE', '11_NDRE', '12_NDRE', '13_NDRE', '14_NDRE', '15_NDRE', '16_NDRE', '17_NDRE', '18_NDRE', '19_NDRE', '20_NDRE', '21_NDRE', '22_NDRE', '23_NDRE', '24_NDRE', '25_NDRE', '26_NDRE', '27_NDRE', '28_NDRE', '29_NDRE', '30_NDRE', '31_NDRE']
inds = range(samples_datesets.size().getInfo())

In [11]:
df = pd.DataFrame(index=inds,columns=cols)
df.head()

,id,0_NDRE,1_NDRE,2_NDRE,3_NDRE,4_NDRE,5_NDRE,6_NDRE,7_NDRE,8_NDRE,...,22_NDRE,23_NDRE,24_NDRE,25_NDRE,26_NDRE,27_NDRE,28_NDRE,29_NDRE,30_NDRE,31_NDRE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data_value = samples_datesets.reduceColumns(
    reducer = ee.Reducer.toList().repeat(33),
    selectors = cols
)
df = pd.DataFrame(data_value.getInfo()['list'],cols)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
id,867.000000,868.000000,869.000000,870.000000,871.000000,872.000000,873.000000,940.000000,941.000000,942.000000,...,857.000000,858.000000,859.000000,860.000000,861.000000,862.000000,863.000000,864.000000,865.000000,866.000000
0_NDRE,0.271618,0.076785,0.138086,0.143267,0.135758,0.115946,0.112281,0.159014,0.228581,0.247534,...,0.350385,0.241264,0.338437,0.270207,0.231967,0.378353,0.396408,0.442837,0.426336,0.468510
1_NDRE,0.272077,0.082208,0.141345,0.144666,0.138434,0.119080,0.114546,0.165336,0.234849,0.254513,...,0.354796,0.247630,0.344330,0.272513,0.240572,0.382450,0.401866,0.448591,0.433858,0.469457
2_NDRE,0.273565,0.088594,0.145235,0.146655,0.141505,0.122631,0.117443,0.172188,0.241744,0.262119,...,0.359629,0.254738,0.350313,0.275466,0.250327,0.386707,0.406857,0.454201,0.441058,0.470152
3_NDRE,0.276073,0.095897,0.149728,0.149217,0.144949,0.126574,0.120951,0.179518,0.249210,0.270299,...,0.364853,0.262538,0.356348,0.279049,0.261160,0.391097,0.411353,0.459628,0.447887,0.470595


In [13]:
df.to_csv('sample_NDRE_2022_1000.csv',encoding='utf-8')